## Introduction
This is an introduction to using FloPy within a model-specific function. If you haven't installed Flopy, go back to the [MODFLOW and FloPy setup notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/MODFLOW%2C%20Python%2C%20and%20FloPy%20Setup.ipynb) and the [FloPy Introduction notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/FloPy%20Introduction.ipynb).

Import the standard libraries

In [ ]:
# Import a few libraries
import sys
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
# import pathlib
# Import the flopy library
import flopy
'flopy' in sys.modules #True

First find where you have your MODFLOW6 executables located on your system.

In [ ]:
# MODFLOW6 executable (only one)
# Executable location of Mf6.exe
# exe_path = "C:\\Hydro\\"
exe_path = "C:\\Hydro\\mf6.4.2\\mf6.4.2_win64\\bin\\"

exe_loc = os.path.dirname(exe_path)
print("Path to MODFLOW 6 executable:", exe_loc)

Let's use the same directory to save the data as the FloPy introduction and then create a path to this workspace. It may be useful to understand your current working directory, this should be whereever you have this notebook saved. You can double check this with the command 'os.getcwd()'.

In [ ]:
# This should return a path to your current working directory
current_directory = os.getcwd()
print(current_directory)

If this is not where you want to save stuff then uncomment the cell below and define the path to establish a new folder and set this to be the new working directory.

In [ ]:
# # define path
# path = pathlib.Path('C:\\Users\\zahas\\Dropbox\\Teaching\\Contaminant hydro 629\\Notebooks_unpublished')
# # if folder doesn't exist then make it 
# path.mkdir(parents=True, exist_ok=True)
# # set working directory to this new folder
# os.chdir(path)
# current_directory = os.getcwd()
# print(current_directory)

In [ ]:
# directory to save data
directory_name = 'data_1D_model'
# Let's add that to the path of the current directory
workdir = os.path.join('.', directory_name)

# if the path exists then we will move on, if not then create a folder with the 'directory_name'
if os.path.isdir(workdir) is False:
    os.mkdir(workdir) 
    print("Directory '% s' created" % workdir) 
else:
    print("Directory '% s' already exists" % workdir) 

Notice however that we don't yet name the folder where we will save data 'dirname'. This will be an input to our model function.


## 1D Model Function
The first thing we do is setup the function. We will use nearly identical settings as we used in the [FloPy Introduction notebook](https://github.com/zahasky/Contaminant-Hydrogeology-Activities/blob/master/FloPy%20Introduction.ipynb) example, but now we are providing a few input variables that can be changed everytime we call the model. The input variables are:

### Function Input:
#### directory name
    direname = 

#### period length 
Time is in selected units, the model time length is the sum of this (for steady state flow it can be set to anything). The format for multi-period input: ```[60., 15*60]```
 
    perlen = 
    
#### advection velocity
Note that this is only an approximate advection flow rate in due to the way that the inlet boundary conditions are being assigned in the MODFLOW BAS6 - Basic Package. More rigorous constraint of constant flux boundaries require the Flow and Head Boundary Package, the Well Package, or the Recharge Package.

    v = 
    
#### dispersivity
Set the longitudinal dispersivity in selected units. What are the units again?

    al = 
    
    

In [ ]:
def model_1D(dirname, perlen, v, al, Cinj=1.0, mixelm=-1, C0=0):
    # start timer to measure how fast the model runs
    tic = time.perf_counter()
    # Model workspace and new sub-directory
    model_ws = os.path.join(workdir, dirname)
    print(model_ws)

    gwfname = 'gwf-' + dirname
     # create the MF6 simulation
    sim = flopy.mf6.MFSimulation(sim_name=dirname, exe_name=os.path.join(exe_loc, 'mf6.exe'), sim_ws=model_ws, 
                                verbosity_level = 2)
    
    # time and length units - use lab units for now
    length_units = "CENTIMETERS"
    time_units = "MINUTES"

    # Modflow stress periods
    # number of stress periods (MF input), calculated from period length input
    nper = len(perlen)
    # nstp (integer) is the number of time steps in a stress period.
    # tsmult (double) is the multiplier for the length of successive time steps. 
    tsmult = 1
    tdis_rc = []
    # loop through perlen and assign period lengths
    for i in range(nper):
        tdis_rc.append((perlen[i], perlen[i]*6, tsmult))
    flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)

    # Instantiating MODFLOW 6 groundwater flow model
    gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True, 
            model_nam_file=f"{gwfname}.nam") 

    # Instantiating MODFLOW 6 solver for flow model
    imsgwf = flopy.mf6.ModflowIms(sim, complexity = "SIMPLE")
    sim.register_ims_package(imsgwf, [gwf.name])
    
    # Model information 
    nlay = 1 # number of layers
    nrow = 1 # number of rows
    ncol = 101 # number of columns
    top = 0 # grid size in direction of Lz
    delc = 4.4 # grid size in direction of Ly, this was choosen such that the model has the same cross-sectional area as the column from the dispersion notebook example
    delr = 0.1 # grid size in direction of Lx
    botm  = -4.4
    
    # length of model in selected units 
    Lx = (ncol - 1) * delr
    print("Model length is: " + str(Lx + delr) + ' ' + str(length_units))

    # Instantiating MODFLOW 6 discretization package
    flopy.mf6.ModflowGwfdis(gwf, length_units=length_units,
            nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc,
            top=top, botm=botm,
            filename=f"{gwfname}.dis")
    
    # hydraulic conductivity
    HK = 1. # what are the units here?
    
    # Instantiating MODFLOW 6 node-property flow package
    flopy.mf6.ModflowGwfnpf(gwf, save_flows=False, icelltype=0,
            k=HK) # k is the hydraulic conductivity 

    # Instantiating MODFLOW 6 initial conditions package for flow model
    # gwf_strt = np.zeros((nlay, nrow, ncol), dtype=float)
    flopy.mf6.ModflowGwfic(gwf, strt=0.0)
    
    # porosity
    prsity = 0.3
    # discharge is based on advection velocity input (again in selected units)
    q = v * prsity
    print("Discharge = " + str(q))
    
    h1 = q * Lx / HK
    print("calculated head differential across column based on provided advection velocity = " + str(h1) )

    # Constant head cells are specified on both ends of the model
    chdspd = [[(0, 0, 0), h1], [(0, 0, ncol - 1), 0.0]]
    # Instantiating MODFLOW 6 constant head package
    flopy.mf6.ModflowGwfchd(gwf, maxbound=len(chdspd), 
                            stress_period_data=chdspd,
                            save_flows=False, pname="CHD1")
    
    # FLow output control
    flopy.mf6.ModflowGwfoc(gwf,
        head_filerecord=f"{gwfname}.hds",
        saverecord=[("HEAD", "ALL")],
        printrecord=[("HEAD", "FIRST"), ("HEAD", "LAST"),])

    #############################################################
    ############### NOW BUILD TRANSPORT #########################
    print(f"Building mf6gwt model in...{model_ws}")
    gwtname = "gwt_" + dirname
    gwt = flopy.mf6.MFModel(sim,
            model_type="gwt6", modelname=gwtname,
            model_nam_file=f"{gwtname}.nam")
    # gwt.name_file.save_flows = True

    # create iterative model solution and register the gwt model with it
    imsgwt = flopy.mf6.ModflowIms(sim, print_option="SUMMARY", linear_acceleration="BICGSTAB",
            filename=f"{gwtname}.ims")
    sim.register_ims_package(imsgwt, [gwt.name])

    # Instantiating MODFLOW 6 transport discretization package
    flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
            delr=delr, delc=delc, top=top, botm=botm,
            filename=f"{gwtname}.dis")

    # Initial conditions
    # initial concentration set to zero everywhere
    # Instantiating MODFLOW 6 transport initial concentrations
    flopy.mf6.ModflowGwtic(gwt, strt=C0, filename=f"{gwtname}.ic")

    # Solute boundary conditions
    # cncspd = [[(0, 0, 0), Cinj]] # constant
    cncspd = {0: [[(0, 0, 0), Cinj]], 1: [[(0, 0, 0), 0]]} # pulse
    # Instantiating MODFLOW 6 transport constant concentration package
    flopy.mf6.ModflowGwtcnc(gwt, maxbound=len(cncspd), stress_period_data=cncspd)

    flopy.mf6.ModflowGwtssm(gwt)
    
    # Mobile Storage and Transfer (MST) Package of the GWT Model for MODFLOW 6 represents solute mass storage, sorption, and frst- or zero-order decay in MOBILE domain.
    flopy.mf6.ModflowGwtmst(gwt, porosity=prsity) # without reactions
    # With reactions
    # Instantiating MODFLOW 6 MST package
    # flopy.mf6.ModflowGwtmst(gwt, sorption='LINEAR', porosity = prsity, 
    #         bulk_density = bulk_density, distcoef = mobile_Kd,
    #         filename=f"{gwtname}.mst")

    # Instantiating MODFLOW 6 transport advection package
    if mixelm == 1:
        scheme = "UPSTREAM"
    elif mixelm == -1:
        scheme = "TVD"
    elif mixelm == 2:
        scheme = "CENTRAL"
    else:
        raise Exception()
    flopy.mf6.ModflowGwtadv(gwt, scheme=scheme)

    # define dispersion/diffusion behavior
    flopy.mf6.ModflowGwtdsp(gwt, xt3d_off=True, alh=al, ath1=al)

    # Instantiating MODFLOW 6 transport output control package
    flopy.mf6.ModflowGwtoc(gwt,
            budget_filerecord=f"{gwtname}.cbc",
            concentration_filerecord=f"{gwtname}.ucn",
            # concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
            saverecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")],
            printrecord=[("CONCENTRATION", "ALL"), ("BUDGET", "ALL")])

    # Instantiating MODFLOW 6 flow-transport exchange mechanism
    flopy.mf6.ModflowGwfgwt(sim,
            exgtype="GWF6-GWT6",
            exgmnamea=gwfname, exgmnameb=gwtname,
            filename=f"{dirname}.gwfgwt")

    # Write simulation
    sim.write_simulation(silent=True)
    success, buff = sim.run_simulation(silent=True)

    # Extract head and concentration field data
    head = gwf.output.head().get_data()
    conc = gwt.output.concentration().get_alldata() # get_data() retrieves only the last timestep
    times = np.array(gwt.output.concentration().get_times())
    
    # Time the function
    toc = time.perf_counter()
    print("Model " + dirname + " ran in " + str(toc-tic) + " seconds")
    print('.')
    
    return conc, head, times

Now lets trying running a model by calling our 'model_1D' function

In [ ]:
dirname = 'run1'
perlen = [60] # min

v = 0.40# cm/min
al = 0.5 # cm
Cinj = 1.0

# Call the FloPy model function
conc, head, times = model_1D(dirname, perlen, v, al, Cinj, mixelm=-1)

In [ ]:
print(times)

Now let's plot the model output as a function of time

In [ ]:
# extract the concentration at the outlet of the model
c_btc = conc[:, 0, 0, -1]
# now plot the time vs the concentration at the outlet
plt.plot(times, c_btc)
plt.xlabel('Time [min]')
plt.ylabel('Concentration')
plt.show()

And as a function of space

In [ ]:
x = np.linspace(0.1, 10., 101)
# Extract the concentration everywhere at the last time index (that is what the -1 means)
c_profile = conc[-1, 0, 0, :]
plt.plot(x, c_profile)
plt.xlabel('Distance from inlet [cm]')
plt.ylabel('Concentration')
plt.ylim([0, 1.1])
plt.show()

# Plot the head profile across the model, before uncommenting this do you have an idea of what the head will look like as a function of space?
# hydraulic_grad = head[0, 0, :]
# # plt.plot(x, hydraulic_grad) 
# plt.xlabel('Distance from inlet [cm]')
# plt.ylabel('Head [cm]')
# plt.show()

## 1D Analytical Solution Function
Now lets compare the numerical results with the analytical solution we looked at in our previous notebook. Note the addition of the finite length outlet boundary conditions with type 1 inlet conditions (Equation A3 in van Genuchtena and Alves, 1982).

In [ ]:
# Remember that we need a few special functions
from scipy.special import erfc as erfc
from scipy.special import erf as erf

# Type 1 inlet conditions, infinite solution
def analytical_model_1D_t1(x, t, v, al):
    # Dispersion
    D = v*al
    # Analytical solution: See lecture slides or (Parker and van Genuchten, 1984) for details
    # Note that the '\' means continued on the next line
    Conc_time_type1 = (1/2)*erfc((x - v*t)/(2*np.sqrt(D*t))) + \
        (1/2)*np.exp(v*x/D)*erfc((x + v*t)/(2*np.sqrt(D*t)))
    
    return Conc_time_type1

# Type 1 inlet conditions, finite length solution
def analytical_model_1D_finite_t1(x, t, v, al, L):
    # Dispersion
    D = v*al
    # Analytical solution: Analytical solution based on Equation A3 in van Genuchtena and Alves, 1982.
    # Note that the '\' means continued on the next line
    Conc_time_type1_finite = (1/2)*erfc((x - v*t)/(2*np.sqrt(D*t))) + \
        (1/2)*np.exp(v*x/D)*erfc((x + v*t)/(2*np.sqrt(D*t))) + \
        (1/2)*(2 + (v*(2*L - x)/D) + v**2*t/D)* \
        np.exp(v*L/D)*erfc(((2*L - x)+ v*t)/(2*np.sqrt(D*t))) - \
        (v**2 *t/(3.1415*D))**(1/2) * np.exp(v*L/D - ((2*L - x + v*t)**2)/(4*D*t))
            
    return Conc_time_type1_finite

# Type 3 inlet conditions, infinite solution
def analytical_model_1D_t3(x, t, v, al):
    # Dispersion
    D = v*al
    # Analytical solution: See lecture slides or (Parker and van Genuchten, 1984 eq 9b) for details
    Conc_time_type3 = (1/2)* erfc((x - v*t)/(2* np.sqrt(D*t))) + \
    np.sqrt((v**2*t)/(3.1415*D))* np.exp(-(x - v*t)**2/(4*D*t)) - \
    (1/2)*(1 + (v*x/D) + (v**2*t/D))* np.exp(v*x/D)* erfc((x + v*t)/(2* np.sqrt(D*t)))
    
    return Conc_time_type3

Call the function to calculate the breakthrough curve at outlet of the core

In [ ]:
# Call the analytical model function
Conc_time_type1 = analytical_model_1D_t1(x[-1], times, v, al)
Conc_time_ftype1 = analytical_model_1D_finite_t1(x[-1], times, v, al, x[-1])
Conc_time_type3 = analytical_model_1D_t3(x[-1], times, v, al)

Now let's plot a comparison.

In [ ]:
plt.figure(figsize=(6, 3), dpi=150)
plt.plot(times, c_btc, label='BTC FloPy')
# plt.plot(times, Conc_time_type1, '--', label='BTC 1D analytical, type 1')
plt.plot(times, Conc_time_ftype1, '--', label='BTC 1D analytical, type 1, finite length')
# # plt.plot(times, Conc_time_type3, '--', label='BTC 1D analytical, type 3')
plt.xlabel('Time [min]');

plt.legend()
plt.show()

Based on these results what is the approximate mean arrival time of the solute at the end of the core? Based on the volumetric flow rate and pore volume of the column does this arrival time make sense? Why or why not?

## Activity:
Using these code, evalute the concration profile along the column a few different times.

In [ ]:
# Choose a timestep to evaluate the analytical solution at
timestep = 60
# Note that this timestep corresponds to the numerical model output increment, the actual model time is given by
print('Model time: ' + str(times[timestep]) + ' min')

# Call the analytical model functions
Conc_time_type1_x = analytical_model_1D_t1(x, times[timestep], v, al)
Conc_time_ftype1_x = analytical_model_1D_finite_t1(x, times[timestep], v, al, x[-1])
Conc_time_type3_x = analytical_model_1D_t3(x, times[timestep], v, al)

In [ ]:
# Extract the concetration profile at a specific timestep
C_profile = conc[timestep, 0, 0, :]

plt.figure(figsize=(6, 3), dpi=150)
plt.plot(x, C_profile, label='Profile FloPy')
plt.plot(x, Conc_time_type1_x, '-', label='Profile 1D analytical, type 1')
plt.plot(x, Conc_time_ftype1_x, '--', label='Profile 1D analytical, type 1, finite length')
plt.plot(x, Conc_time_type3_x, '--', label='Profile 1D analytical, type 3')
plt.xlabel('Distance from inlet [cm]')
plt.legend()
plt.show()

This is showing the solute front concentration as a function of space. Do we call this a concentration profile or breakthrough curve?

In [ ]:
# Now repeat this for a later time
timestep = 150 # change to whatever 
print('Model time: ' + str(times[timestep]/60) + ' min')

# Call the analytical model function
# copy and paste code, work through each line to make sure you understand what each command does



#### Work through the following questions:
Explain the differences between these concentration profiles at different times. 

What is the mechanism that leads to differences in solute front location? For example, if time and spatial location don't change, what will cause the green line to shift left or shift right along the x-axis?

How is the slope of the line changing at the two different times? Does the slope increase or decrease with time? Why?

Why does the FloPy model fit a Type 1 inlet boundary condition and not a Type 3 (hint go back and examine that package used to set solute boundary conditions in the model)? Do the differences between the two solutions increase or decrease with changes in dispersivity?